In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [13]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

openai_llm = ChatOpenAI(
    model="gpt-4.1-nano"
)

groq_llm = ChatGroq(
    model="llama-3.3-70b-versatile"
)

openai_llm, groq_llm

(ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117e9a0d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117e98b90>, root_client=<openai.OpenAI object at 0x117e99950>, root_async_client=<openai.AsyncOpenAI object at 0x117e9a490>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********')),
 ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x117e99d10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x117e98f50>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')))

In [14]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    product_name: str = Field(description="Name of the product")
    product_description: str = Field(description="A comprehensive overview of the product, including its core features, unique selling points, materials or technology used, intended use cases, target audience, and any competitive advantages it offers.")
    product_price: float = Field(description="Price of the product")

In [15]:
from langchain_core.output_parsers import JsonOutputParser

JsonOutputParser(pydantic_object=Product)

JsonOutputParser(pydantic_object=<class '__main__.Product'>)

In [16]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import SystemMessage

parser = JsonOutputParser(pydantic_object=Product)

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            f"You are a helpful assistant good at describing products. {parser.get_format_instructions()}"
        )
    ),
    HumanMessagePromptTemplate.from_template("{input}")
])

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are a helpful assistant good at describing products. The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"product_name": {"description": "Name of the product", "title": "Product Name", "type": "string"}, "product_description": {"description": "A comprehensive overview of the product, including its core features, unique selling points, materials or technology used, intended use cases, target audience, and any competitive advantages it offers.", "title": 

In [17]:
chain = prompt | openai_llm | parser
print(chain.invoke({"input": "Tell me about the MacBook Air"}))

print("\n\n")

chain = prompt | groq_llm | parser
print(chain.invoke({"input": "Tell me about the MacBook Air"}))

{'product_name': 'MacBook Air', 'product_description': 'The MacBook Air is a sleek and lightweight laptop designed by Apple, renowned for its portability, stunning Retina display, and powerful performance. It features the latest Apple Silicon M2 chip, ensuring fast processing and efficient energy use. The laptop boasts a thin aluminum body, a Magic Keyboard for comfortable typing, and a Force Touch trackpad. Ideal for students, professionals, and on-the-go users, the MacBook Air offers long battery life, high-quality audio, and seamless integration within the Apple ecosystem, making it a top choice for productivity and entertainment.', 'product_price': 999}



{'product_name': 'MacBook Air', 'product_description': 'The MacBook Air is a line of ultra-portable laptops designed by Apple. It features a sleek and lightweight design, making it ideal for working on-the-go. The laptop is equipped with a high-resolution Retina display, powerful processors, and ample storage options. The MacBook

In [18]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import YamlOutputParser
from langchain_core.messages import SystemMessage

parser = YamlOutputParser(pydantic_object=Product)

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            f"You are a helpful assistant good at describing products. {parser.get_format_instructions()}"
        )
    ),
    HumanMessagePromptTemplate.from_template("{input}")
])

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are a helpful assistant good at describing products. The output should be formatted as a YAML instance that conforms to the given JSON schema below.\n\n# Examples\n## Schema\n```\n{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}\n```\n## Well formatted instance\n```\n- name: John Doe\n  avg: 0.3\n- name: Jane Maxfield\n  avg: 1.4\n```\n\n## Schema\n```\n{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type"

In [19]:
chain = prompt | openai_llm | parser
print(chain.invoke({"input": "Tell me about the MacBook Air"}))

print("\n\n")

chain = prompt | groq_llm | parser
print(chain.invoke({"input": "Tell me about the MacBook Air"}))

product_name='MacBook Air' product_description='The MacBook Air is a sleek and lightweight laptop designed by Apple, featuring a high-resolution Retina display, powerful M2 chip for swift performance, and all-day battery life. It is constructed with a durable aluminum unibody and incorporates innovative features such as a Magic Keyboard, Touch ID for secure authentication, and Thunderbolt ports for versatile connectivity. Ideal for students, professionals, and creatives, the MacBook Air offers exceptional portability without compromising on performance, making it perfect for both everyday tasks and more demanding workflows. The device also emphasizes eco-friendly manufacturing practices, aligning with sustainability efforts, and provides a seamless integration within the Apple ecosystem.\n' product_price=999.0



product_name='MacBook Air' product_description='The MacBook Air is a line of ultra-portable laptops developed by Apple, known for their sleek design, lightweight construction,